In [1]:
import findspark
import os
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f

# os.environ["SPARK_HOME"] = "/home/my_home/spark-3.5.2-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

findspark.init()

spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.driver.memory", "2G")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .getOrCreate()
)

print("Spark session is now available")

24/09/16 11:37:38 WARN Utils: Your hostname, nsphung-ia resolves to a loopback address: 127.0.1.1; using 10.163.133.235 instead (on interface enx64c901a5e8d3)
24/09/16 11:37:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 11:37:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session is now available


In [2]:
# Find from data/output/result.json the journal that mention drugs the most
journals: DataFrame = spark.read.option("multiline", "true").json(
    "../data/output/result.json"
)
journals

atccode,clinical_trials,drug,journals,pubmeds
A01AD,"[{27 April 2020, ...",EPINEPHRINE,"[{01/02/2020, The...","[{01/03/2020, 8, ..."
S03AA,"[{NULL, NULL, NUL...",TETRACYCLINE,"[{01/01/2020, Jou...","[{2020-01-01, 6, ..."
V03AB,"[{NULL, NULL, NUL...",ETHANOL,"[{2020-01-01, Psy...","[{2020-01-01, 6, ..."
A04AD,"[{1 January 2020,...",DIPHENHYDRAMINE,"[{01/01/2019, Jou...","[{02/01/2019, 3, ..."
6302001,"[{NULL, NULL, NUL...",ISOPRENALINE,"[{NULL, NULL}]","[{NULL, NULL, NUL..."
A03BA,"[{NULL, NULL, NUL...",ATROPINE,"[{NULL, NULL}]","[{NULL, NULL, NUL..."
R01AD,"[{1 January 2020,...",BETAMETHASONE,"[{NULL, NULL}, {1...","[{NULL, NULL, NUL..."
HELLO,"[{NULL, NULL, NUL...",CORONA,"[{NULL, NULL}]","[{NULL, NULL, NUL..."


In [3]:
journals.select(
    "atccode", "drug", f.explode("journals.journal").alias("journal")
).where(f.col("journal").isNotNull()).show(30, False)

+-------+---------------+-----------------------------------------------------------+
|atccode|drug           |journal                                                    |
+-------+---------------+-----------------------------------------------------------+
|A01AD  |EPINEPHRINE    |The journal of allergy and clinical immunology. In practice|
|A01AD  |EPINEPHRINE    |The journal of allergy and clinical immunology. In practice|
|A01AD  |EPINEPHRINE    |Journal of emergency nursing                               |
|A01AD  |EPINEPHRINE    |Journal of emergency nursing\xc3\x28                       |
|S03AA  |TETRACYCLINE   |Journal of food protection                                 |
|S03AA  |TETRACYCLINE   |Psychopharmacology                                         |
|S03AA  |TETRACYCLINE   |American journal of veterinary research                    |
|V03AB  |ETHANOL        |Psychopharmacology                                         |
|A04AD  |DIPHENHYDRAMINE|Journal of emergency nursing 

In [4]:
journals.select(
    "atccode", "drug", f.explode("journals.journal").alias("journal")
).where(f.col("journal").isNotNull()).groupBy("journal").agg(
    f.count("atccode").alias("numberOfDrugs")
).sort(
    f.col("numberOfDrugs").desc()
).show(
    20, False
)

+-----------------------------------------------------------+-------------+
|journal                                                    |numberOfDrugs|
+-----------------------------------------------------------+-------------+
|Journal of emergency nursing                               |3            |
|The journal of allergy and clinical immunology. In practice|2            |
|Psychopharmacology                                         |2            |
|Journal of food protection                                 |1            |
|Hôpitaux Universitaires de Genève                          |1            |
|Journal of emergency nursing\xc3\x28                       |1            |
|The Journal of pediatrics                                  |1            |
|American journal of veterinary research                    |1            |
+-----------------------------------------------------------+-------------+



In [5]:
journals.select(
    "atccode", "drug", f.explode("journals.journal").alias("journal")
).where(f.col("journal").isNotNull()).sort("drug").show(60, False)

+-------+---------------+-----------------------------------------------------------+
|atccode|drug           |journal                                                    |
+-------+---------------+-----------------------------------------------------------+
|R01AD  |BETAMETHASONE  |Hôpitaux Universitaires de Genève                          |
|A04AD  |DIPHENHYDRAMINE|Journal of emergency nursing                               |
|A04AD  |DIPHENHYDRAMINE|The Journal of pediatrics                                  |
|A04AD  |DIPHENHYDRAMINE|Journal of emergency nursing                               |
|A01AD  |EPINEPHRINE    |The journal of allergy and clinical immunology. In practice|
|A01AD  |EPINEPHRINE    |The journal of allergy and clinical immunology. In practice|
|A01AD  |EPINEPHRINE    |Journal of emergency nursing                               |
|A01AD  |EPINEPHRINE    |Journal of emergency nursing\xc3\x28                       |
|V03AB  |ETHANOL        |Psychopharmacology           